## Demonstração - Pytorch

## Um exemplo mais completo

### Carrega os dados e os separa em treino e teste

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"

dfPropaganda = pd.read_csv('Advertising.csv',index_col=0)

y= dfPropaganda.loc[:,'Sales']
X = dfPropaganda.loc[:,['TV','Radio','Newspaper']]

y_tensor = torch.tensor(y.to_numpy())
X_tensor = torch.tensor(X.to_numpy())

X_treino, X_teste, y_treino, y_teste = train_test_split(X_tensor, y_tensor, test_size = 0.30, random_state=5)

print(X_treino.shape)
print(y_treino.shape)

print(X_teste.shape)
print(y_teste.shape)
print(y_teste.ndim)
print(X_treino.dtype)

#Manda para o dispositivo selecionado - no caso, GPU
X_treino = X_treino.float().to(device)
y_treino = y_treino.float().to(device)

#reshape com -1: infere o valor com base no que está sendo passado (passo para sair de 1d e ir para 2d e evitar problemas de cálculos).
y_treino = torch.reshape(y_treino, (-1,1))

X_teste = X_teste.float().to(device)
y_teste = y_teste.float().to(device)
y_teste = torch.reshape(y_teste, (-1,1))

print("\nDepois da conversão para float e do reshape")
print(X_treino.dtype)
print(y_treino.shape)
print(y_teste.shape)
print(y_teste.ndim)

#Já convertemos para float, pois inicialmente o padrão é double'

torch.Size([140, 3])
torch.Size([140])
torch.Size([60, 3])
torch.Size([60])
1
torch.float64

Depois da conversão para float e do reshape
torch.float32
torch.Size([140, 1])
torch.Size([60, 1])
2


### Implementa uma classe com o modelo

Para definir uma rede neural em PyTorch, criamos uma classe que herda de nn.Module. Definimos as camadas da rede na função __init__ e especificamos como os dados passarão pela rede na função forward. <br>


In [2]:
class Feedforward(torch.nn.Module):
    
        #Esta função é onde você define as camadas totalmente conectadas em sua rede neural
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            
            
            self.input_size = input_size
            self.hidden_size  = hidden_size
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc4 = torch.nn.Linear(self.hidden_size, 1)
            
        
        #Forward: Especifica como os dados passarão pelo seu modelo
        #x representa nossos dados
        def forward(self, x):
            
            output = self.fc1(x)
            output = F.relu(output)
            
            
            output = self.fc2(output)
            output = F.relu(output)
            
            output = self.fc3(output)
            output = F.relu(output)
            
            output = self.fc4(output)

            return output

### Modelo e critério de otimização
Para acelerar as operações na rede neural, nós a movemos para a GPU, se disponível.

In [3]:
#input = 3 (número de features), e hidden size = 10 (número de neurôneos na camada escondida)
model = Feedforward(3, 10).to(device)
print(model)
criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

Feedforward(
  (fc1): Linear(in_features=3, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=1, bias=True)
)


### Treino e teste do modelo

In [4]:
model.eval()
y_pred = model(X_teste)
antes_treino = criterion(y_pred, y_teste) 
print('Teste - perda antes do treinamento' , antes_treino.item())


#-----------------Treinamento
model.train()
epoch = 1000

for epoch in range(epoch):
    
    #Explicitamente configura os gradientes em zero
    optimizer.zero_grad()
    
    # Passe Forward
    y_pred = model(X_treino)
    
    # Computa a perda
    loss = criterion(y_pred, y_treino)
    
    print('Epoch {}: perda treino: {}'.format(epoch, loss.item()))
    
    # Propaga os erros (backpropagation) e atualiza os pesos
    loss.backward()
    optimizer.step()

    
    
#-----------------Avaliação
model.eval()

y_pred = model(X_teste)
after_train = criterion(y_pred, y_teste) 
print('Teste - perda depois do treinamento' , after_train.item())    


Teste - perda antes do treinamento 162.53648376464844
Epoch 0: perda treino: 165.1116485595703
Epoch 1: perda treino: 162.56381225585938
Epoch 2: perda treino: 159.9998321533203
Epoch 3: perda treino: 157.4158477783203
Epoch 4: perda treino: 154.81414794921875
Epoch 5: perda treino: 152.19459533691406
Epoch 6: perda treino: 149.5593719482422
Epoch 7: perda treino: 146.9037322998047
Epoch 8: perda treino: 144.23013305664062
Epoch 9: perda treino: 141.53173828125
Epoch 10: perda treino: 138.81961059570312
Epoch 11: perda treino: 136.08837890625
Epoch 12: perda treino: 133.3384246826172
Epoch 13: perda treino: 130.57290649414062
Epoch 14: perda treino: 127.79534149169922
Epoch 15: perda treino: 125.00003051757812
Epoch 16: perda treino: 122.19566345214844
Epoch 17: perda treino: 119.38546752929688
Epoch 18: perda treino: 116.57852935791016
Epoch 19: perda treino: 113.76547241210938
Epoch 20: perda treino: 110.95496368408203
Epoch 21: perda treino: 108.14932250976562
Epoch 22: perda treino

Epoch 296: perda treino: 2.5820472240448
Epoch 297: perda treino: 2.571320056915283
Epoch 298: perda treino: 2.5601847171783447
Epoch 299: perda treino: 2.549793243408203
Epoch 300: perda treino: 2.539794683456421
Epoch 301: perda treino: 2.529709815979004
Epoch 302: perda treino: 2.519573450088501
Epoch 303: perda treino: 2.5104167461395264
Epoch 304: perda treino: 2.501518964767456
Epoch 305: perda treino: 2.492147445678711
Epoch 306: perda treino: 2.4832751750946045
Epoch 307: perda treino: 2.474832057952881
Epoch 308: perda treino: 2.4662892818450928
Epoch 309: perda treino: 2.457653045654297
Epoch 310: perda treino: 2.449505567550659
Epoch 311: perda treino: 2.441560745239258
Epoch 312: perda treino: 2.433539628982544
Epoch 313: perda treino: 2.4259250164031982
Epoch 314: perda treino: 2.4181108474731445
Epoch 315: perda treino: 2.410048484802246
Epoch 316: perda treino: 2.401942729949951
Epoch 317: perda treino: 2.3940470218658447
Epoch 318: perda treino: 2.3863914012908936
Epoch

Epoch 587: perda treino: 1.3128864765167236
Epoch 588: perda treino: 1.3097063302993774
Epoch 589: perda treino: 1.3061543703079224
Epoch 590: perda treino: 1.3020429611206055
Epoch 591: perda treino: 1.298601508140564
Epoch 592: perda treino: 1.2950336933135986
Epoch 593: perda treino: 1.2912155389785767
Epoch 594: perda treino: 1.2876416444778442
Epoch 595: perda treino: 1.2841469049453735
Epoch 596: perda treino: 1.280950665473938
Epoch 597: perda treino: 1.2775447368621826
Epoch 598: perda treino: 1.2740744352340698
Epoch 599: perda treino: 1.2706279754638672
Epoch 600: perda treino: 1.2672464847564697
Epoch 601: perda treino: 1.263906717300415
Epoch 602: perda treino: 1.260399580001831
Epoch 603: perda treino: 1.2569527626037598
Epoch 604: perda treino: 1.253549337387085
Epoch 605: perda treino: 1.2502951622009277
Epoch 606: perda treino: 1.2480523586273193
Epoch 607: perda treino: 1.2435345649719238
Epoch 608: perda treino: 1.2415549755096436
Epoch 609: perda treino: 1.2376598119

Epoch 877: perda treino: 0.561084508895874
Epoch 878: perda treino: 0.5593702793121338
Epoch 879: perda treino: 0.5576016306877136
Epoch 880: perda treino: 0.5558968782424927
Epoch 881: perda treino: 0.5543555021286011
Epoch 882: perda treino: 0.5528401136398315
Epoch 883: perda treino: 0.5512795448303223
Epoch 884: perda treino: 0.5499662160873413
Epoch 885: perda treino: 0.5494145154953003
Epoch 886: perda treino: 0.5487712621688843
Epoch 887: perda treino: 0.5477101802825928
Epoch 888: perda treino: 0.5461524724960327
Epoch 889: perda treino: 0.5447512865066528
Epoch 890: perda treino: 0.5436328053474426
Epoch 891: perda treino: 0.5429383516311646
Epoch 892: perda treino: 0.542476236820221
Epoch 893: perda treino: 0.542234480381012
Epoch 894: perda treino: 0.5415703654289246
Epoch 895: perda treino: 0.5421638488769531
Epoch 896: perda treino: 0.5412397384643555
Epoch 897: perda treino: 0.5410507917404175
Epoch 898: perda treino: 0.5383231043815613
Epoch 899: perda treino: 0.53605890